# IMPORTANT!! 
## Make sure you have run the preprocessing script (`preprocessing/hog/main.py`) so that you have the necessary HOG data.

---

# KNN with HOG

Adding our own packages to the path so that they can be found.

In [40]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Needed packages

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from preprocessing.hog import Flatten

## Loading Metadata

We load the hog metadata file in order to find the paths to the training and testing data.

In [42]:
hog_metadata_path = '../../data/hog/metadata.csv'
hog_metadata = pd.read_csv(hog_metadata_path, index_col=0)
hog_metadata.channels = hog_metadata.channels.astype(str)

## Grid Search for KNN

In [43]:
train_metadata = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == channel)]
train = np.load(f'../../{train_metadata .data_path.values[0]}')
train_label = np.load(f'../../{train_metadata.label_path.values[0]}')

val_metadata = hog_metadata[(hog_metadata.type == 'dev') & (hog_metadata.channels == channel)]
val = np.load(f'../../{val_metadata.data_path.values[0]}')
val_label = np.load(f'../../{val_metadata.label_path.values[0]}')

parameters = {'n_neighbors':range(3, 15)}
knn = KNeighborsClassifier()
gs = GridSearchCV(knn, parameters)

pipeline = Pipeline([('flatten', Flatten()), ('gs', gs)])

pipeline.fit(np.vstack((train, val)), np.hstack((train_label, val_label)))

gs.best_params_

{'n_neighbors': 3}

## Creating the KNN Pipeline

KNN expects a 2D array, so we need to flatten the image dimensions from `(samples, width, height)` into `(samples, width * height)`. Our custom `Flatten` class does this job for us.

In [44]:
pipeline = Pipeline([('flatten', Flatten()), ('knn', KNeighborsClassifier(n_neighbors=3))])

## Getting the data, fitting, scoring

We go through all of the channels, and fetch the train and test data associated with each. Then we fit, predict, and save the resulting confusion matrix.

In [45]:
confusion_matrices = {}

In [46]:
for channel in hog_metadata.channels.unique():
    train_metadata = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == channel)]
    train = np.load(f'../../{train_metadata .data_path.values[0]}')
    train_label = np.load(f'../../{train_metadata.label_path.values[0]}')

    test_metadata = hog_metadata[(hog_metadata.type == 'test') & (hog_metadata.channels == channel)]
    test = np.load(f'../../{test_metadata.data_path.values[0]}')
    test_label = np.load(f'../../{test_metadata.label_path.values[0]}')
    
    pipeline.fit(train, train_label)
    
    confusion_mat = confusion_matrix(test_label, pipeline.predict(test))
    confusion_matrices[channel] = confusion_mat

## Confusion Matrix

In [47]:
display_labels = ['No Glacier', 'Glacier']

In [48]:
fig, axes = plt.subplots(ncols=3, nrows=1, sharex=True, sharey=True, figsize=(12, 4))

axes[0].set_title("Channels: 1, 2, 3")
axes[1].set_title("Channels: 6, 7, 8")
axes[2].set_title("Channels: 11, 12, 13")

ConfusionMatrixDisplay(confusion_matrices['[1, 2, 3]'], display_labels=display_labels).plot(ax=axes[0])
ConfusionMatrixDisplay(confusion_matrices['[6, 7, 8]'], display_labels=display_labels).plot(ax=axes[1])
ConfusionMatrixDisplay(confusion_matrices['[11, 12, 13]'], display_labels=display_labels).plot(ax=axes[2]);